In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

# Cargar el archivo Excel
file_path = '/content/drive/MyDrive/BID/ULTIMOS/colombia/input/cultivos_colombia.xlsx'
data = pd.read_excel(file_path)

# Cambiar los nombres de las columnas
columnas_renombradas = {
    'Área sembrada (ha)': 'ha',
    'Área cosechada (ha)': 'haC',
    'Producción (t)': 'Ton',
    'Rendimiento (t/ha)': 'Ton/ha'
}
data = data.rename(columns=columnas_renombradas)

# Eliminar filas con valores nulos en columnas clave
data = data.dropna(subset=['Departamento', 'Cultivo', 'Año'])

# Filtrar los estados especificados
selected_states = [
    'PUTUMAYO', 'CAQUETÁ', 'AMAZONAS', 'GUAINÍA', 'GUAVIARE', 'VAUPÉS'
]
filtered_data = data[data['Departamento'].isin(selected_states)].copy()

# Reemplazar las etiquetas en la columna 'Cultivo'
traducciones = {
    "PALMA DE ACEITE": "Oil palm",
    "MAÍZ": "Corn",
    "ARROZ": "Rice",
    "SOYA": "Soy",
    "PLÁTANO": "Plantain",
    "CAÑA": "Sugarcane",
    "YUCA": "Yuca",
    "CACAO": "Cocoa",
    "MARAÑÓN": "Cashew",
    "CAFÉ": "Coffee"
}
filtered_data.loc[:, 'Cultivo'] = filtered_data['Cultivo'].replace(traducciones)

# Eliminar registros duplicados
filtered_data = filtered_data.drop_duplicates()

# Filtrar los datos para el año 2023
data_2023 = filtered_data[filtered_data['Año'] == 2023]

# Obtener los 10 cultivos principales por área sembrada en 2023
top_10_cultivos_2023 = data_2023.groupby('Cultivo')['ha'].sum().nlargest(10)

# Crear una lista de los 10 cultivos principales
cultivos_principales = top_10_cultivos_2023.index.tolist()

# Filtrar el dataframe original para incluir solo los 10 cultivos principales en el año 2023
filtered_data_top_10 = data_2023[data_2023['Cultivo'].isin(cultivos_principales)].copy()

# Generar la tabla consolidada con rendimiento ponderado
tabla_consolidada = filtered_data_top_10.groupby('Cultivo').agg({
    'ha': 'sum',
    'haC': 'sum',
    'Ton': 'sum'
}).reset_index()
# Calcular rendimiento ponderado
tabla_consolidada['Ton/ha'] = tabla_consolidada['Ton'] / tabla_consolidada['haC']

# Generar la tabla por departamento con cultivos como columnas y las etiquetas de área sembrada, área cosechada, producción y rendimiento en una misma columna
melted_data = filtered_data_top_10.melt(
    id_vars=['Departamento', 'Cultivo'],
    value_vars=['ha', 'haC', 'Ton', 'Ton/ha'],
    var_name='Indicador',
    value_name='Valor'
)
tabla_por_departamento = melted_data.pivot_table(
    index=['Departamento', 'Indicador'],
    columns='Cultivo',
    values='Valor',
    aggfunc='sum'
).reset_index()

# Generar la tercera tabla con departamentos como columnas y los cultivos en otra columna
tabla_por_cultivo = melted_data.pivot_table(
    index=['Cultivo', 'Indicador'],
    columns='Departamento',
    values='Valor',
    aggfunc='sum'
).reset_index()

# Exportar las tablas a un archivo Excel
with pd.ExcelWriter('/content/drive/MyDrive/BID/ULTIMOS/colombia/output/tablas_cultivos.xlsx') as writer:
    tabla_consolidada.to_excel(writer, sheet_name='Tabla Consolidada', index=False)
    tabla_por_departamento.to_excel(writer, sheet_name='Tabla por Departamento', index=False)
    tabla_por_cultivo.to_excel(writer, sheet_name='Tabla por Cultivo', index=False)

print("Las tablas se han exportado correctamente a 'tablas_cultivos_nuevo.xlsx'")


Las tablas se han exportado correctamente a 'tablas_cultivos_nuevo.xlsx'
